# <div align="center"><b> Procesamiento de los recortes </b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> potery source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [ ]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

from modulo_apps.config import config as CONFIG
from loguru import logger as LOGGER

import modulo_apps.labeling.procesador_recortes as ProcesadorRecortes
import modulo_apps.s3_comunication.procesador_s3 as ProcesadorS3
import modulo_apps.labeling.procesador_anotaciones_mongodb as ProcesadorAnotacionesMongoDB

import cv2 as cv

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [ ]:
LOGGER.info("Configuración cargada correctamente.")

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Procesamiento de los recortes.                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Notebook encargado de realizar el procesamiento de los recortes. Esto implica actualizar los recortes realizados en Minio de las anotaciones</small>|

</div>

## Tabla de contenidos

1. [Procesamiento de los recortes](#procesamiento-de-los-recortes)

## 1. [Procesamiento de los recortes](#procesamiento-de-los-recortes)

In [ ]:
# TODO: Implementar esto con prefect.

Flujo de datos:
1. Descargar el dataset de todas las imágenes y sus anotaciones.
2. Recortar las imágenes según las anotaciones.
3. Subir los recortes a Minio.

```mermaid
flowchart TD
    A[Descargar dataset] --> B[Recortar imágenes]
    B --> C[Subir recortes a Minio]
```

In [8]:
ProcesadorS3.download_image_from_minio("8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm")

2025-06-27 17:34:43.821 | DEBUG    | modulo_apps.s3_comunication.procesador_s3:download_image_from_minio:50 - Imagen 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm descargada correctamente en downloads\images\8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm.jpg.


WindowsPath('downloads/images/8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm.jpg')

In [11]:
ProcesadorAnotacionesMongoDB.download_annotations_as_coco_from_mongodb(images_names=["8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm"])

2025-06-27 17:38:05.685 | WARNING  | modulo_apps.labeling.procesador_anotaciones_mongodb:_create_images_fields:257 - No se encontraron anotaciones para el parche 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_1.
2025-06-27 17:38:05.686 | WARNING  | modulo_apps.labeling.procesador_anotaciones_mongodb:_create_images_fields:257 - No se encontraron anotaciones para el parche 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_3.
2025-06-27 17:38:05.687 | DEBUG    | modulo_apps.labeling.procesador_anotaciones_mongodb:_download_annotation_as_coco_from_mongodb:365 - Archivo JSON guardado en downloads\coco_annotations\cvat_8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_annotations.json.
2025-06-27 17:38:05.694 | DEBUG    | modulo_apps.labeling.procesador_anotaciones_mongodb:_load_coco_annotation_from_mongodb:416 - Anotaciones cargadas desde downloads\coco_annotations\cvat_8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_annotations.json.

WindowsPath('downloads/coco_annotations/cvat_images_annotations.json')

In [17]:
image = cv.imread("downloads/images/8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm.jpg")
coco_annotations = ProcesadorAnotacionesMongoDB.load_coco_annotations_from_mongodb(images_names=["8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm"])
ProcesadorRecortes.cut_palms_from_image(image, coco_annotations, "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm")

2025-06-27 17:45:24.844 | WARNING  | modulo_apps.labeling.procesador_anotaciones_mongodb:_create_images_fields:257 - No se encontraron anotaciones para el parche 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_1.
2025-06-27 17:45:24.844 | WARNING  | modulo_apps.labeling.procesador_anotaciones_mongodb:_create_images_fields:257 - No se encontraron anotaciones para el parche 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_3.
2025-06-27 17:45:24.845 | DEBUG    | modulo_apps.labeling.procesador_anotaciones_mongodb:_download_annotation_as_coco_from_mongodb:365 - Archivo JSON guardado en downloads\coco_annotations\cvat_8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_annotations.json.
2025-06-27 17:45:24.847 | DEBUG    | modulo_apps.labeling.procesador_anotaciones_mongodb:_load_coco_annotation_from_mongodb:416 - Anotaciones cargadas desde downloads\coco_annotations\cvat_8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_annotations.json.

WindowsPath('downloads/cutouts/8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm')

In [18]:
ProcesadorRecortes.upload_cutouts_to_mino()

2025-06-27 17:54:11.231 | DEBUG    | modulo_apps.labeling.procesador_recortes:clear_minio_cutouts_folder:193 - No se encontraron objetos para eliminar con el prefijo recortes/8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm/.
2025-06-27 17:54:11.260 | DEBUG    | modulo_apps.labeling.procesador_recortes:upload_cutouts_to_mino:236 - Imagen 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_cutout_1.jpg subida a MinIO.
2025-06-27 17:54:11.280 | DEBUG    | modulo_apps.labeling.procesador_recortes:upload_cutouts_to_mino:236 - Imagen 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_cutout_2.jpg subida a MinIO.
2025-06-27 17:54:11.299 | DEBUG    | modulo_apps.labeling.procesador_recortes:upload_cutouts_to_mino:236 - Imagen 8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_cutout_3.jpg subida a MinIO.
2025-06-27 17:54:11.312 | DEBUG    | modulo_apps.labeling.procesador_recortes:upload_cutouts_to_mino:236 - Imagen 8deOctubreyCentenario-EspLibreLarranaga_